In [15]:
import numpy as np
import pandas as pd
import os

In [16]:
!pwd

/data/HCP_preproc/7T_movie/movie_cpm/code


In [17]:
subj_list = np.load('subj_list.npy', allow_pickle=True)
print(len(subj_list))
subj_list[0:10]

176


array(['100610', '102311', '102816', '104416', '105923', '108323',
       '109123', '111514', '114823', '115017'], dtype=object)

In [18]:
f_suffix = "Movement_RelativeRMS_mean"

run_list_rest = ["REST1_7T_PA", "REST4_7T_AP"]
run_list_movie = ["MOVIE1_7T_AP", "MOVIE2_7T_PA", "MOVIE3_7T_PA", "MOVIE4_7T_AP"]

In [19]:
run_names_short=[run for sublist in [run_list_rest, run_list_movie] for run in sublist]

columns = []
for run in run_names_short:
    columns.append(run + "_" + f_suffix)

motion_data = pd.DataFrame(index=subj_list, columns=columns)

Change the directories below to where you have the HCP data saved:

In [20]:
data_path_rest = "/data/HCP_preproc/resting_7T_1.6mm_59k_functional_preprocessed_Jan2020/"
data_path_movie = "/data/HCP/HCP_7T/7T_Movie_fMRI_Preproc/"

for subj in subj_list:
    
    for run in run_list_rest:
        f_name = data_path_rest + subj + "/MNINonLinear/Results/rfMRI_" + run + "/" + f_suffix + ".txt"
        col_name = run + "_" + f_suffix
        
        if os.path.isfile(f_name):
            with open(f_name) as file:
                tmp = file.read().splitlines()[0]
                motion_data.loc[subj, col_name] = tmp
        else:
            print("File missing for {} {}".format(subj, run))
            motion_data.loc[subj, col_name] = np.nan
            
    for run in run_list_movie:
        f_name = data_path_movie + subj + "/MNINonLinear/Results/tfMRI_" + run + "/" + f_suffix + ".txt"
        col_name = run + "_" + f_suffix
        
        if os.path.isfile(f_name):
            with open(f_name) as file:
                tmp = file.read().splitlines()[0]
                motion_data.loc[subj, col_name] = tmp
        else:
            print("File missing for {} {}".format(subj, run))
            motion_data.loc[subj, col_name] = np.nan

In [21]:
motion_data.index.name = 'Subject'
motion_data.head()

,REST1_7T_PA_Movement_RelativeRMS_mean,REST4_7T_AP_Movement_RelativeRMS_mean,MOVIE1_7T_AP_Movement_RelativeRMS_mean,MOVIE2_7T_PA_Movement_RelativeRMS_mean,MOVIE3_7T_PA_Movement_RelativeRMS_mean,MOVIE4_7T_AP_Movement_RelativeRMS_mean
Subject,,,,,,
100610,0.18972,0.202442,0.257122,0.178953,0.201716,0.240824
102311,0.0525035,0.139191,0.0832096,0.0789687,0.103597,0.09436
102816,0.176665,0.144463,0.228877,0.176782,0.156451,0.14872
104416,0.0334869,0.049917,0.0394932,0.0263406,0.0332663,0.0474593
105923,0.0892302,0.0854001,0.0676684,0.0531432,0.0572334,0.0604693


### Do individual clips and matched rest blocks

In [22]:
video_tr_lookup = pd.read_csv('../data/video_tr_lookup.csv')
video_tr_lookup.head()

,clipno_overall,run,clipno_in_run,clip_name,start_tr,stop_tr
0,1,MOVIE1_7T_AP,1,twomen,20,265
1,2,MOVIE1_7T_AP,2,bridgeville,285,506
2,3,MOVIE1_7T_AP,3,pockets,526,714
3,4,MOVIE1_7T_AP,4,overcome,735,798
4,5,MOVIE1_7T_AP,5,testretest1,818,901


In [23]:
data_path_motion = "/data/HCP/HCP_7T/7T_Movie_fMRI_Preproc/"
f_suffix = "Movement_RelativeRMS"

In [24]:
clip_to_rest_name_dict = {
        "twomen": "REST1_7T_PA",
        "bridgeville": "REST1_7T_PA",
        "pockets": "REST1_7T_PA",
        "overcome": "REST1_7T_PA",
        "testretest1": "REST1_7T_PA",

        "inception": "REST1_7T_PA",
        "socialnet": "REST1_7T_PA",
        "oceans": "REST1_7T_PA",
        "testretest2": "REST1_7T_PA",

        "flower": "REST4_7T_AP",
        "hotel": "REST4_7T_AP",
        "garden": "REST4_7T_AP",
        "dreary": "REST4_7T_AP",
        "testretest3": "REST4_7T_AP",

        "homealone": "REST4_7T_AP",
        "brokovich": "REST4_7T_AP",
        "starwars": "REST4_7T_AP",
        "testretest4": "REST4_7T_AP",
        }

In [ ]:
clip_list = video_tr_lookup.clip_name.tolist()

for clip in clip_list:
    # Get start and stop TRs
    start_tr = video_tr_lookup.loc[video_tr_lookup["clip_name"]==clip, "start_tr"].values[0]
    stop_tr = video_tr_lookup.loc[video_tr_lookup["clip_name"]==clip, "stop_tr"].values[0]
    
    # Read in motion file for this run
    run_name = video_tr_lookup.query('clip_name==@clip')["run"].tolist()[0]
    col_name = clip + "_Movement_RelativeRMS_mean"
    for subj in subj_list:
        f_name = data_path_motion + subj + "/MNINonLinear/Results/tfMRI_" + run_name + "/" + f_suffix + ".txt"
        tmp = np.loadtxt(f_name)
        tmp = tmp[start_tr:stop_tr]
        motion_data.loc[subj, col_name] = np.mean(tmp)
        
    # Read in motion file for matched REST run
    run_name = clip_to_rest_name_dict[clip]
    col_name = clip + "_SameRestBlock_Movement_RelativeRMS_mean"
    for subj in subj_list:
        f_name = data_path_rest + subj + "/MNINonLinear/Results/rfMRI_" + run_name + "/" + f_suffix + ".txt"
        tmp = np.loadtxt(f_name)
        tmp = tmp[start_tr:stop_tr]
        motion_data.loc[subj, col_name] = np.mean(tmp)

In [ ]:
motion_data.head()

### Save

In [ ]:
motion_data.to_csv("../data/motion_data.csv")